In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
from selenium import webdriver

In [2]:
# Create connection variable
conn = 'mongodb://localhost:27017'

# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)

# Connect to a database. Will create one if not already available.
db = client.mars_db

# Drops collection if available to remove duplicates
db.mars_collection.drop()

db.mars_collection


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_db'), 'mars_collection')

In [3]:
# empty dictionary to store all values in
mars = {}

# NASA Mars News

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
# SELENIUM
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
driver = webdriver.Chrome()
response = driver.get(url)
soup = bs(driver.page_source, "html.parser")


In [7]:
# find title and paragraph
#title = soup.find('div', class_='content_title')
#mars['title']=title.text

#paragraph = soup.find('div', class_='article_teaser_body')
#mars['paragraph'] = paragraph
#print(mars)

ti='//*[@id="page"]/div[2]/div/article/div/section/div/ul/li[1]/div/div/div[2]'
mars['title'] = driver.find_element_by_xpath(ti).text

para = '//*[@id="page"]/div[2]/div/article/div/section/div/ul/li[1]/div/div/div[3]'
mars['paragraph'] = driver.find_element_by_xpath(para).text
print(mars)

{'title': "A Year of Surprising Science From NASA's InSight Mars Mission", 'paragraph': "A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet."}


# JPL Mars Space Images - Featured Image

-Visit the url for JPL Featured Space Image here.
-Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
-Make sure to find the image url to the full size .jpg image.
-Make sure to save a complete url string for this image.

In [8]:
!which chromedriver

/usr/local/bin/chromedriver


In [9]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

browser.click_link_by_partial_text('FULL IMAGE')
image = soup.find('a', class_="button fancybox")
image_url = image['data-fancybox-href']
firstparturl = 'https://www.jpl.nasa.gov/'
mars['featured_image_url'] = firstparturl + image_url
print(mars)
browser.quit()

# Mars Weather

Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [ ]:
# URL of MARS WEATHER page to be scraped
url = "https://twitter.com/marswxreport?lang=en"

# retrieve page with requests module
response = requests.get(url)

# create Beautiful Soup object and parse
soup = bs(response.text, 'lxml')
#print(soup.prettify())

In [ ]:
# scrape latest weather tweet
mars_weather = soup.find('p', class_='TweetTextSize')
mars['mars_weather'] = mars_weather.text
print(mars)

In [ ]:
# remove text from a tag


# Mars Facts

Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
Use Pandas to convert the data to a HTML table string.

In [ ]:
# save url of MARS FACTS webpage
url = 'https://space-facts.com/mars/'

In [ ]:
# extract tables from website
tables = pd.read_html(url)
tables

In [ ]:
# create df
mars_df = tables[0]
mars_df.columns=['Category','MARS']


In [ ]:
# convert df to html table string 
mars_html = mars_df.to_html()
mars['mars_html'] = mars_html.replace('\n','')
print(mars)

In [ ]:
# save html to file and hide index
mars_df.to_html('mars_facts.html', index=False)
!open mars_facts.html

# Mars Hemispheres

*Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
*You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
*Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
*Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [ ]:
# SELENIUM
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver = webdriver.Chrome()
response = driver.get(url)
soup = bs(driver.page_source, "html.parser")


In [ ]:
hemisphere_image_urls = []
y = 0

# print title of each hemisphere in for loop
for each in list(soup.find_all('div', class_='item')):
    title = each.find('h3').text
    
    for x in range(4):
        x += y
        driver.find_element_by_xpath(f'//*[@id="product-section"]/div[2]/div[{x+1}]/a/img').click()    
        driver.find_element_by_xpath('//*[@id="wide-image-toggle"]').click()

        # print urls
        img = driver.find_element_by_xpath('//*[@id="wide-image"]/div/ul/li[1]/a')
        img_url = img.get_attribute('href')

        hemispheres = {"title": title, "img_url": img_url}
        hemisphere_image_urls.append(hemispheres)
        break
        
    driver.back()
    y += 1    

mars['hemisphere_image_urls']=hemisphere_image_urls
print(mars)
driver.quit()

In [ ]:
print(list(mars))
